In [167]:
import requests
from requests_html import HTML
import pandas as pd
import time

In [168]:
base_url = 'https://stackoverflow.com/questions/tagged/'
tag = 'python'
query_condition = 'Votes'
# https://stackoverflow.com/questions/tagged/python?tab=Votes
url =f'{base_url}{tag}?tab={query_condition}'

In [169]:
def parse_data(html):
    summary_stats = html.find(".js-post-summary-stats")
    summary_content = html.find(".s-post-summary--content")
    posts_lst = []

    for i in range(len(summary_content)):
        post_dict = {}
        post_dict["votes"] = summary_stats[i].text.split("\n")[0].split("v")[0]
        post_dict["answers"] = summary_stats[i].text.split("\n")[1].split("a")[0]
        post_dict["views"] = summary_stats[i].text.split("\n")[2].split("v")[0]
        posts_dict["question_title"] = summary_content[i].text.split("\n")[0]
        post_dict["question"] = summary_content[i].text.split("\n")[1]
        post_dict["tags"] = summary_content[i].text.split("\n")[2]
        post_dict["user"] = summary_content[i].text.split("\n")[4]
        post_dict["user_reputation"] = summary_content[i].text.split("\n")[5]
        post_dict["time_asked"] = summary_content[i].text.split("\n")[-1]

        posts_lst.append(post_dict)
        
    return posts_lst

In [170]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200,300):
        return []
    html_text = r.text
    html = HTML(html=html_text)
    data = parse_data(html)
    return data

In [171]:
def scrape_tag(tag='python', query_filter='Votes', max_pages=50, pagesize=25):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    data_ = []
    for p in range(max_pages):
        page_num = p + 1
        #     https://stackoverflow.com/questions/tagged/python?tab=votes&page=5&pagesize=15
        url = f'{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}'
        data_ += extract_data_from_url(url)
        time.sleep(1.2)
    return data_

In [190]:
questions_data = scrape_tag(tag='python', query_filter='Votes', max_pages=50, pagesize=25)

In [192]:
questions_df = pd.DataFrame(questions_data)
drop_data = (questions_df['time_asked'] == 'Devoted') | (questions_df['user_reputation'] == 'Devoted') | (questions_df['user'] == 'Devoted')
questions_df = questions_df[~drop_data].reset_index(drop=True)
questions_df.head()

,votes,answers,views,question,tags,user,user_reputation,time_asked
0,11879,47,2.8m,What is the use of the yield keyword in Python...,python iterator generator yield coroutine,Alex. S.,134k,"asked Oct 23, 2008 at 22:21"
1,6652,24,981k,"In Python, what are metaclasses and what do we...",python oop metaclass python-class python-datam...,e-satis,544k,"asked Sep 19, 2008 at 6:10"
2,6545,42,4.8m,"How do I check whether a file exists or not, w...",python file file-exists,spence91,72k,"asked Sep 17, 2008 at 12:55"
3,6165,56,2.6m,"I have two Python dictionaries, and I want to ...",python dictionary merge,Carl Meyer,113k,"asked Sep 2, 2008 at 7:44"
4,5622,63,4.0m,How do you call an external command (as if I'd...,python shell terminal subprocess command,freshWoWer,57.9k,"asked Sep 18, 2008 at 1:35"


In [193]:
questions_df.shape

(2498, 8)

In [194]:
questions_df.to_csv('stackoverflow_questions.csv', index=False)